In [6]:
import pandas as pd
import plotly.graph_objects as go

In [7]:
#from google.colab import files
#uploaded = files.upload()

In [8]:
data=pd.read_excel('TotalReturn10_5.xlsx',skiprows=5,names=['Date','Price'],index_col='Date',parse_dates=['Date'])
data=data.reindex(data.index.sort_values(ascending=True))

In [9]:
monthly_data=data.resample('M').last()

In [10]:
monthly_data['Return']=monthly_data['Price'].pct_change()*100
monthly_data

,Price,Return
Date,,
1991-12-31,104.03,NaN
1992-01-31,101.57,-2.364702
1992-02-29,102.02,0.443044
1992-03-31,101.14,-0.862576
1992-04-30,101.85,0.701997
...,...,...
2021-11-30,517.85,0.809827
2021-12-31,516.20,-0.318625
2022-01-31,505.84,-2.006974


In [11]:
monthly_data.sort_values(by='Return').head(10).index

DatetimeIndex(['2003-07-31', '2022-03-31', '2004-04-30', '2009-12-31',
               '2016-11-30', '2009-01-31', '1999-02-28', '2010-12-31',
               '1994-03-31', '2002-03-31'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [289]:
n=int(input('How many monthly data points do you want:'))
fig=go.Figure()
fig.add_trace(go.Bar(x=monthly_data.sort_values(by='Return').head(n).index,y=monthly_data['Return'].sort_values().head(n)))
fig.show()

How many monthly data points do you want:10


In [13]:
print(f"{monthly_data.sort_values(by='Return').index[0].year} - {monthly_data.sort_values(by='Return').index[0].month}"),
round(monthly_data['Return'].sort_values().iloc[0],2)

2003 - 7


-4.95

In [14]:
fig=go.Figure()
def SetColor(x):
  if(x<-4):
    return "red"
  elif x<-1:
    return "yellow"
  else:
    return "green"

fig.add_trace(go.Bar(x0=0,dx=1,text=monthly_data.sort_values(by='Return').head(10).index.strftime("%m - %Y <br> %b"),y=monthly_data['Return'].sort_values().head(10),marker=dict(color=list(map(SetColor,monthly_data['Return'].sort_values().head(10))))))
fig.update_layout(title='Top 10 Worst Monthly Performance <br> <i>(from left to right)</i> ')
fig.show()

#marker={"size": 25, "color": list(range(-3,10)), "cmid": 0}

In [15]:
fig=go.Figure()
def SetColorOp(x):
  if(x>4):
    return "green"
  elif x>2:
    return "yellow"
  else:
    return "red"

fig.add_trace(go.Bar(x0=0,dx=1,text=monthly_data.sort_values(by='Return',ascending=False).head(10).index.strftime("%m - %Y <br> %b"),y=monthly_data['Return'].sort_values(ascending=False).head(10),marker=dict(color=list(map(SetColorOp,monthly_data['Return'].sort_values(ascending=False).head(10))))))
fig.update_layout(title='Top 10 Best Monthly Performance <br> <i>(from left to right)</i> ')
fig.show()


In [16]:
#quarterly performance
quarterly_data=data.resample('Q').last()
quarterly_data['Return']=quarterly_data['Price'].pct_change()*100
fig=go.Figure()
fig.add_trace(go.Bar(x=quarterly_data.sort_values(by='Return').head(10).index,y=quarterly_data['Return'].sort_values().head(10)))
fig.show()

In [17]:
fig=go.Figure()
def SetColor(x):
  if(x<-4):
    return "red"
  elif x<-1:
    return "yellow"
  else:
    return "green"

fig.add_trace(go.Bar(x0=0,dx=1,text=quarterly_data.sort_values(by='Return').head(10).index.strftime("%m - %Y <br> %b"),y=quarterly_data['Return'].sort_values().head(10),marker=dict(color=list(map(SetColor,quarterly_data['Return'].sort_values().head(10))))))
fig.update_layout(title='Top 10 Worst Quarterly Performance <br> <i>(from left to right)</i> ')
fig.show()

In [18]:
fig=go.Figure()
def SetColorOp(x):
  if(x>4):
    return "green"
  elif x>2:
    return "yellow"
  else:
    return "red"

fig.add_trace(go.Bar(x0=0,dx=1,text=quarterly_data.sort_values(by='Return',ascending=False).head(10).index.strftime("%m - %Y <br> %b"),y=quarterly_data['Return'].sort_values(ascending=False).head(10),marker=dict(color=list(map(SetColorOp,quarterly_data['Return'].sort_values(ascending=False).head(10))))))
fig.update_layout(title='Top 10 Best Quarterly Performance <br> <i>(from left to right)</i> ')
fig.show()

In [20]:
#own timeframe
def specific_timeframe(timeframe):
  specific=data.resample(timeframe).last()
  specific['Return']=specific['Price'].pct_change()*100
  fig=go.Figure()
  fig.add_trace(go.Bar(x=specific.sort_values(by='Return').head(10).index,y=specific['Return'].sort_values().head(10)))
  fig.show()

In [40]:
x,y=monthly_data.describe()['Return'].loc[['mean','std']]
x,y

(0.43627581777940133, 1.566628196891589)

In [99]:
import scipy.stats as stats
mean,std = monthly_data.describe()['Return'].loc[['mean','std']]
# Calculate a z-score from a provided mean and standard deviation
print(f"{monthly_data.iloc[-1].name.strftime('%b%y')} return should happen in every other {(1/stats.norm.cdf(x=monthly_data['Return'].iloc[-1],loc=mean,scale=std)):0.00f} months")
print(f"z-score is {(monthly_data['Return'].iloc[-1]-mean)/std:0.02f}")


Mar22 return should happen in every other 511 months
z-score is -2.88


In [145]:
def monthly_probabilities(date):
  
  if(monthly_data['Return'].loc[date][0]>mean):    
    print(f"{monthly_data.loc[date].index[0].strftime('%b%y')} return should happen in every other {(1/(1-stats.norm.cdf(x=monthly_data['Return'].loc[date][0],loc=mean,scale=std))):0.00f} months")
    print(f"z-score is {(monthly_data['Return'].loc[date][0]-mean)/std:0.02f}")
    print(f"Probability to have better outcome is {1-stats.norm.cdf(x=monthly_data['Return'].loc[date][0],loc=mean,scale=std)}")
  else:
    print(f"{monthly_data.loc[date].index[0].strftime('%b%y')} return should happen in every other {(1/stats.norm.cdf(x=monthly_data['Return'].loc[date][0],loc=mean,scale=std)):0.00f} months")
    print(f"z-score is {(monthly_data['Return'].loc[date][0]-mean)/std:0.02f}")
    print(f"Probability to have worse outcome is {stats.norm.cdf(x=monthly_data['Return'].loc[date][0],loc=mean,scale=std)}")
monthly_probabilities('2022-03')

Mar22 return should happen in every other 511 months
z-score is -2.88
Probability to have worse outcome is 0.0019571959925438562


In [195]:

def quarterly_probabilities(date):
  q_mean,q_std = quarterly_data.describe()['Return'].loc[['mean','std']]

  if(quarterly_data['Return'].loc[date][0]>q_mean):    
    print(f"{quarterly_data.loc[date].index[0].strftime('%b%y')} return should happen in every other {(1/(1-stats.norm.cdf(x=quarterly_data['Return'].loc[date][0],loc=q_mean,scale=q_std))):0.00f} quarters")
    print(f"z-score is {(quarterly_data['Return'].loc[date][0]-q_mean)/q_std:0.02f}")
    print(f"Probability to have better outcome is {1-stats.norm.cdf(x=quarterly_data['Return'].loc[date][0],loc=q_mean,scale=q_std)}")
  else:
    print(f"{quarterly_data.loc[date].index[0].strftime('%b%y')} return should happen in every other {(1/stats.norm.cdf(x=quarterly_data['Return'].loc[date][0],loc=q_mean,scale=q_std)):0.00f} quarters")
    print(f"z-score is {(quarterly_data['Return'].loc[date][0]-q_mean)/q_std:0.02f}")
    print(f"Probability to have worse outcome is {stats.norm.cdf(x=quarterly_data['Return'].loc[date][0],loc=q_mean,scale=q_std)}")
quarterly_probabilities('2022-03')


Mar22 return should happen in every other 131 quarters
z-score is -2.43
Probability to have worse outcome is 0.007607070992384179


In [276]:
#not ideal, issues with downsampling. works for months+
def own_probability(timeline,date):
  unique_data=data.resample(timeline).last()
  unique_data['Return']=unique_data['Price'].pct_change()*100

  o_mean,o_std = unique_data.describe()['Return'].loc[['mean','std']]

  if(unique_data['Return'].loc[date][0]>o_mean):    
    print(f"{unique_data.loc[date].index[0].strftime('%b%y')} return should happen in every other {(1/(1-stats.norm.cdf(x=unique_data['Return'].loc[date][0],loc=o_mean,scale=o_std))):0.00f} {timeline}")
    print(f"z-score is {(unique_data['Return'].loc[date][0]-o_mean)/o_std:0.02f}")
    print(f"Probability to have better outcome is {1-stats.norm.cdf(x=unique_data['Return'].loc[date][0],loc=o_mean,scale=o_std)}")
  else:
    print(f"{unique_data.loc[date].index[0].strftime('%b%y')} return should happen in every other {(1/stats.norm.cdf(x=unique_data['Return'].loc[date][0],loc=o_mean,scale=o_std)):0.00f} {timeline}")
    print(f"z-score is {(unique_data['Return'].loc[date][0]-o_mean)/o_std:0.02f}")
    print(f"Probability to have worse outcome is {stats.norm.cdf(x=unique_data['Return'].loc[date][0],loc=o_mean,scale=o_std)}")
  
own_probability('Y','2022')


Dec22 return should happen in every other 21 Y
z-score is -1.67
Probability to have worse outcome is 0.04738097016247564


In [192]:
safe=data.resample('Y').last().pct_change().describe().loc[['mean','std']]
(data.resample('Y').last().tail().pct_change().iloc[-1]-safe.iloc[0])/safe.iloc[1]
#safe
#data.resample('Y').last().pct_change().iloc[-1]#.tail().pct_change().iloc[-1]

Price   -1.670796
dtype: float64

In [196]:
p=stats.norm.cdf(x=monthly_data['Return'].iloc[-1],loc=mean,scale=std) #calculating probability
z_score = stats.norm.ppf(p) #calculating z-score from probability
1/stats.norm.cdf(z_score) #event to happen every other 510th month

510.93503349158954

510.93503349158954

In [197]:
#comparing this month/quarter vs previous data

In [261]:
monthly_data.iloc[-1]['Return']
#entire pandemic returns
monthly_data.loc['2020-01':'2021-12'].mean() #average monthly return during 2020-2021 is 0.24% (annualized that is 2.88% )
monthly_data.loc['2020-01':].describe() #since the start of COVID the average return has been negative.

,Price,Return
count,27.000000,27.000000
mean,520.742222,-0.021152
std,13.171522,1.537860
min,483.410000,-4.083414
25%,513.875000,-0.725006
50%,517.850000,0.110512
75%,532.220000,0.648413
max,538.180000,3.083298


In [274]:
colors_hist=['blue' for i in monthly_data.loc['2020-01':].index]
colors_hist[-1]='blue'

fig=go.Figure()
fig.add_trace(go.Histogram(x=monthly_data.loc['2020-01':]['Return'],xbins=dict(size=0.05,start=-8,end=8),marker_color=colors_hist))#,x='Return',nbins=10))#,xbins=dict(size=1)))
fig.update_xaxes(range=(-8,8))
fig.show()


In [298]:
data.resample('W').last().isna().sum()/data.resample('W').last().count()
#pretty good chunk of the weekly data is missing (it was reported monthly back in the day)

Price    0.158474
dtype: float64

In [374]:
from numpy import timedelta64
#data['Delta']=data.iloc[1].name-data.iloc[0].name
#data[data['Delta']<31]
#data['Delta']=
data['Delta']=data.index
data['Delta']=data['Delta'].shift(periods=1)
data['Delta']=data['Delta']-data.index
data.dropna(inplace=True)
weekly_data=data[data['Delta']>'-10 days']

In [375]:
weekly_data=weekly_data.drop(columns=['Delta'])

In [376]:
weekly_data=weekly_data.resample('W').last().pct_change()
weekly_data

,Price
Date,
1997-05-25,NaN
1997-06-01,0.006626
1997-06-08,0.008799
1997-06-15,0.004727
1997-06-22,0.003645
...,...
2022-03-06,0.016474
2022-03-13,-0.018709
2022-03-20,-0.011700


In [379]:
weekly_data.describe().loc[['mean','std']]

,Price
mean,0.000942
std,0.007524


In [398]:
worst_weeks=weekly_data.sort_values(by='Price',ascending=True).head(10)#.loc['2022']
#how many weeks from 2022 are in the top 10 worst weeks:
print(f"there are {worst_weeks.loc['2022'].count().values[0]} weeks from 2022 in the top 10 worst weeks")
print(f"that is {(worst_weeks.loc['2022'].count()/worst_weeks.count()).values[0]*100}% of the top 10 worst weeks")

there are 1 weeks from 2022 in the top 10 worst weeks
that is 10.0% of the top 10 worst weeks
